In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pandas as pd


In [ ]:
class DrivingDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)  # e.g. columns: [filename, steer, speed]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')

        steer = torch.tensor(row['steer'], dtype=torch.float32)
        speed = torch.tensor(row['speed'], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor([steer, speed])

In [ ]:
# 2️⃣ 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# 3️⃣ 모델 정의
class DrivingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_ftrs, 2)  # steer, speed

    def forward(self, x):
        return self.backbone(x)

In [ ]:
# 4️⃣ 학습 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DrivingNet().to(device)

criterion = nn.MSELoss()  # 회귀 문제
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# 5️⃣ 데이터 로더
train_dataset = DrivingDataset('train.csv', 'train_images', transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# 6️⃣ 학습 루프
for epoch in range(10):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), 'driving_model.pth')